In [ ]:
!nvidia-smi

Wed Feb 14 11:45:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00


In [ ]:
import os
import random
import torch
from datasets import load_dataset
from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, )
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import pipeline

In [ ]:
system_message = "Ciao! Sono un risolutore di un gioco italiano in cui dato un insieme di 5 parole (clues), devo trovare la soluzione che consiste in una sesta parola che deve essere legata ad ognuna delle 5 parole date in input. Le 5 parole in input non sono legate tra loro. \nPer le soluzioni terrò conto dei proverbi italiani, dei modi di dire italiani e della cultura italiana e internazionale"

#esperto di linguistica che date 5 parole in input, estrae la parola italiana che più si associa logicamente ad ognuna di queste 5 parole "
print(system_message)

Ciao! Sono un risolutore di un gioco italiano in cui dato un insieme di 5 parole (clues), devo trovare la soluzione che consiste in una sesta parola che deve essere legata ad ognuna delle 5 parole date in input. Le 5 parole in input non sono legate tra loro. 
Per le soluzioni terrò conto dei proverbi italiani, dei modi di dire italiani e della cultura italiana e internazionale


In [ ]:
with open('/content/drive/MyDrive/nlp4fun_train_v2.xml', 'r') as f:
    content = f.read()

In [ ]:
!pip install -q xmltodict

In [ ]:
import xmltodict

data = xmltodict.parse(content)
print(data)

{'games': {'game': [{'id': 'a22f1a30-8908-48c3-92b8-fb801d25191f', 'clue': ['pane', 'gioco', 'testa', 'osso', 'zoccolo'], 'solution': 'duro', 'type': 'TV'}, {'id': 'e01e7994-ccdf-4767-89d6-d288a2d5cb50', 'clue': ['verde', 'nave', 'gioco', 'pomodoro', 'collo'], 'solution': 'bottiglia', 'type': 'TV'}, {'id': '3fc953bd-bd48-4fb9-a86c-bd979c8b5b4f', 'clue': ['spalla', 'giacomo', '77', 'kessler', 'bugie'], 'solution': 'gambe', 'type': 'boardgame'}, {'id': 'bd0fd97e-7498-495b-bd82-a810f6e6bc91', 'clue': ['dita', 'corso', 'basso', 'spaghetti', 'cassa'], 'solution': 'chitarra', 'type': 'TV'}, {'id': 'bb58e30b-3bbd-4844-8bed-26984057d746', 'clue': ['remo', 'fornaio', 'altare', "fico d'india", 'elica'], 'solution': 'pala', 'type': 'boardgame'}, {'id': '07304ea8-aa8c-4bf1-b7a1-5d9004535fc4', 'clue': ['porta', 'ferrara', 'via', 'rosa', 'disegno'], 'solution': 'foglio', 'type': 'TV'}, {'id': 'f48ddf70-5c1d-4d08-8dd6-5c419d66818c', 'clue': ['baseball', 'posta', 'lama', 'giorno', 'stile'], 'solution'

In [ ]:
data = data['games']['game']
data[:5]

[{'id': 'a22f1a30-8908-48c3-92b8-fb801d25191f',
  'clue': ['pane', 'gioco', 'testa', 'osso', 'zoccolo'],
  'solution': 'duro',
  'type': 'TV'},
 {'id': 'e01e7994-ccdf-4767-89d6-d288a2d5cb50',
  'clue': ['verde', 'nave', 'gioco', 'pomodoro', 'collo'],
  'solution': 'bottiglia',
  'type': 'TV'},
 {'id': '3fc953bd-bd48-4fb9-a86c-bd979c8b5b4f',
  'clue': ['spalla', 'giacomo', '77', 'kessler', 'bugie'],
  'solution': 'gambe',
  'type': 'boardgame'},
 {'id': 'bd0fd97e-7498-495b-bd82-a810f6e6bc91',
  'clue': ['dita', 'corso', 'basso', 'spaghetti', 'cassa'],
  'solution': 'chitarra',
  'type': 'TV'},
 {'id': 'bb58e30b-3bbd-4844-8bed-26984057d746',
  'clue': ['remo', 'fornaio', 'altare', "fico d'india", 'elica'],
  'solution': 'pala',
  'type': 'boardgame'}]

In [ ]:
print(f"Sono registrate {len(data)} partite della Ghigliottina")
print(f"di cui {len([element for element in data if element['type'] == 'TV'])} appartengono al gioco in TV")
print(f"e {len([element for element in data if element['type'] == 'boardgame'])} appartengono al gioco da tavolo")

Sono registrate 315 partite della Ghigliottina
di cui 204 appartengono al gioco in TV
e 111 appartengono al gioco da tavolo


In [ ]:
import pandas as pd

dataset = pd.DataFrame.from_dict(data, orient='columns', dtype=None, columns=None)
dataset = dataset[['clue', 'solution']]
dataset.clue = dataset.clue.apply(lambda x: ", ".join(x))
dataset

,clue,solution
0,"pane, gioco, testa, osso, zoccolo",duro
1,"verde, nave, gioco, pomodoro, collo",bottiglia
2,"spalla, giacomo, 77, kessler, bugie",gambe
3,"dita, corso, basso, spaghetti, cassa",chitarra
4,"remo, fornaio, altare, fico d'india, elica",pala
...,...,...
310,"tutti, cardinale, pericolo, mille, mattina",uno
311,"amico, rosso, lente, cintura, terra",fuoco
312,"dentifricio, pantaloni, tv, cilindro, non capire",tubo
313,"marcia, bianco, metà, comune, testimone",matrimonio


In [ ]:
train_df = dataset.sample(frac=0.9, random_state=42)
test_df = dataset.drop(train_df.index)

In [ ]:
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

#Step 8. Load the Llama 2 Model

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "/content/train.jsonl"

new_model = "ghigliottina-llama-2-7b"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

output_dir = "./results"

num_train_epochs = 30

fp16 = False
bf16 = False

per_device_train_batch_size = 4

per_device_eval_batch_size = 4
gradient_accumulation_steps = 1

gradient_checkpointing = True

max_grad_norm = 0.3


learning_rate = 2e-4
weight_decay = 0.001

optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"

max_steps = -1

warmup_ratio = 0.03

group_by_length = True

save_steps = 0

logging_steps = 25

max_seq_length = None
packing = False

device_map = {"": 0} ## LOAD THE ENTIRE MODLE ON THE GPU

Load the dataset and Train

In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files='/content/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\nInput:' + clue + ' \nOutput: [/INST] ' + solution for clue, solution in zip(examples['clue'], examples['solution'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\nInput:' + clue + ' \nOutput: [/INST] ' + solution for clue, solution in zip(examples['clue'], examples['solution'])]}, batched=True)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/284 [00:00<?, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/284 [00:00<?, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,3.612600
50,1.509700
75,0.491200
100,0.378100
125,0.375300
150,0.370200
175,0.334200
200,0.356800
225,0.333900
250,0.325600


## Testing the Fine Tuned Llama2 LLM


In [ ]:
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nInput: 'messa', 'vetro', 'lingua', 'calore', 'trasporto' \nOutput: [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(prompt)
print(result[0]['generated_text'])

[INST] <<SYS>>
Ciao! Sono un risolutore di un gioco italiano in cui dato un insieme di 5 parole (clues), devo trovare la soluzione che consiste in una sesta parola che deve essere legata ad ognuna delle 5 parole date in input. Le 5 parole in input non sono legate tra loro. 
Per le soluzioni terrò conto dei proverbi italiani, dei modi di dire italiani e della cultura italiana e internazionale
<</SYS>>

Input: 'messa', 'vetro', 'lingua', 'calore', 'trasporto' 
Output: [/INST] lasciare 
buonanotte 
modo di fare a which will be the output?

Hint: (buonanotte) 

Input: buonanotte, matita, carta, punto, staffa 
Output: [/./] 


In [ ]:
del model

In [ ]:
del pipe

NameError: name 'pipe' is not defined

In [ ]:
del trainer

In [ ]:
import gc
gc.collect()
gc.collect()

0

In [ ]:
import torch

# Release GPU memory
torch.cuda.empty_cache()

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
#model.save_pretrained(model_path)
#tokenizer.save_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/llama-2-7b-ghigliottina_30epoch-FineTuned"  # change to your preferred path

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NotImplementedError: You are calling `save_pretrained` on a 4-bit converted model. This is currently not supported